In [1]:
import h2o
import numpy as np
import pandas as pd
from h2o.estimators.random_forest import H2ORandomForestEstimator
from h2o.estimators.gbm import H2OGradientBoostingEstimator
h2o.init()
h2o.cluster().show_status()

Checking whether there is an H2O instance running at http://localhost:54321..... not found.
Attempting to start a local H2O server...
  Java Version: java version "1.8.0_171"; Java(TM) SE Runtime Environment (build 1.8.0_171-b11); Java HotSpot(TM) 64-Bit Server VM (build 25.171-b11, mixed mode)
  Starting server from /usr/local/lib/python3.6/dist-packages/h2o/backend/bin/h2o.jar
  Ice root: /tmp/tmpvib04c72
  JVM stdout: /tmp/tmpvib04c72/h2o_rishav_started_from_python.out
  JVM stderr: /tmp/tmpvib04c72/h2o_rishav_started_from_python.err
  Server is running at http://127.0.0.1:54321
Connecting to H2O server at http://127.0.0.1:54321... successful.


H2O cluster uptime:,01 secs
H2O cluster timezone:,Asia/Kolkata
H2O data parsing timezone:,UTC
H2O cluster version:,3.18.0.10
H2O cluster version age:,4 days
H2O cluster name:,H2O_from_python_rishav_bfo0hm
H2O cluster total nodes:,1
H2O cluster free memory:,1.681 Gb
H2O cluster total cores:,4
H2O cluster allowed cores:,4
H2O cluster status:,"accepting new members, healthy"


H2O cluster uptime:,01 secs
H2O cluster timezone:,Asia/Kolkata
H2O data parsing timezone:,UTC
H2O cluster version:,3.18.0.10
H2O cluster version age:,4 days
H2O cluster name:,H2O_from_python_rishav_bfo0hm
H2O cluster total nodes:,1
H2O cluster free memory:,1.681 Gb
H2O cluster total cores:,4
H2O cluster allowed cores:,4
H2O cluster status:,"accepting new members, healthy"


In [2]:
df = h2o.import_file("Claims1.csv")

Parse progress: |█████████████████████████████████████████████████████████| 100%


In [3]:
cols = list(df.columns)
cols.pop(cols.index('FraudFound_P'))
df = df[cols+['FraudFound_P']]

In [4]:
df[32] = df[32].asfactor()
predictors = df.columns[0:32]
response = 'FraudFound_P'

In [5]:
# split into train and validation sets
train, test, valid = df.split_frame(ratios=[.7, .15], seed = 1234)
#train, valid = df.split_frame(ratios = [.8], seed = 1234)

In [6]:
train[0:-1].head()

WeekOfMonth,WeekOfMonthClaimed,Age,PolicyNumber,RepNumber,Deductible,DriverRating,Year,Month,DayOfWeek,Make,AccidentArea,DayOfWeekClaimed,MonthClaimed,Sex,MaritalStatus,Fault,PolicyType,VehicleCategory,VehiclePrice,Days_Policy_Accident,Days_Policy_Claim,PastNumberOfClaims,AgeOfVehicle,AgeOfPolicyHolder,PoliceReportFiled,WitnessPresent,AgentType,NumberOfSuppliments,AddressChange_Claim,NumberOfCars,BasePolicy
3,4,34,2,15,400,4,1994,0,2,6,1,0,0,1,0,0,4,1,5,4,3,0,5,4,0,0,1,0,0,0,1
5,2,47,3,7,400,3,1994,9,4,6,1,3,10,1,1,0,4,1,5,4,3,1,6,6,1,0,1,0,0,0,1
5,2,27,5,3,400,1,1994,0,0,6,1,1,1,0,0,1,4,1,5,4,3,0,4,4,1,0,1,0,0,0,1
4,1,20,6,12,400,3,1994,9,4,6,1,2,10,1,0,1,4,1,5,4,3,0,4,2,1,0,1,2,0,0,1
1,3,36,7,14,400,1,1994,1,5,6,1,0,1,1,1,1,4,1,5,4,3,1,6,5,1,0,1,1,0,0,1
1,4,0,8,1,400,4,1994,10,4,6,1,1,2,1,0,0,4,1,5,4,3,1,0,0,1,0,1,0,0,0,1
4,5,30,9,7,400,4,1994,11,5,6,1,2,11,1,0,0,4,1,5,4,3,0,5,4,1,1,1,2,0,0,1
2,3,71,11,7,400,3,1994,2,6,9,1,2,2,1,0,0,1,0,5,4,3,0,7,8,1,0,1,0,0,0,0
5,5,52,12,13,400,1,1994,2,0,6,1,0,2,1,1,0,2,1,1,4,3,2,7,6,1,0,1,0,0,0,2
3,3,28,13,11,400,1,1994,0,4,5,1,4,0,1,1,0,2,1,5,4,3,1,6,4,1,0,1,0,0,0,2


In [7]:
X_train = train[0:-1].as_data_frame().as_matrix()
y_train = train[-1].as_data_frame().as_matrix()

In [8]:
from imblearn.over_sampling import SMOTE
# Apply SMOTE's
kind = 'regular'
sm = SMOTE(kind='regular')
X_res, y_res = sm.fit_sample(X_train, y_train)

/usr/local/lib/python3.6/dist-packages/sklearn/utils/validation.py:578: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [10]:
p = np.c_[X_res,y_res]
train = pd.DataFrame(p, columns = ['WeekOfMonth', 'WeekOfMonthClaimed', 'Age',
       'PolicyNumber', 'RepNumber', 'Deductible', 'DriverRating', 'Year',
       'Month', 'DayOfWeek', 'Make', 'AccidentArea', 'DayOfWeekClaimed',
       'MonthClaimed', 'Sex', 'MaritalStatus', 'Fault', 'PolicyType',
       'VehicleCategory', 'VehiclePrice', 'Days_Policy_Accident',
       'Days_Policy_Claim', 'PastNumberOfClaims', 'AgeOfVehicle',
       'AgeOfPolicyHolder', 'PoliceReportFiled', 'WitnessPresent', 'AgentType',
       'NumberOfSuppliments', 'AddressChange_Claim', 'NumberOfCars',
       'BasePolicy', 'FraudFound_P'])

In [11]:
train.head()

,WeekOfMonth,WeekOfMonthClaimed,Age,PolicyNumber,RepNumber,Deductible,DriverRating,Year,Month,DayOfWeek,...,AgeOfVehicle,AgeOfPolicyHolder,PoliceReportFiled,WitnessPresent,AgentType,NumberOfSuppliments,AddressChange_Claim,NumberOfCars,BasePolicy,FraudFound_P
0,3.0,4.0,34.0,2.0,15.0,400.0,4.0,1994.0,0.0,2.0,...,5.0,4.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0
1,5.0,2.0,47.0,3.0,7.0,400.0,3.0,1994.0,9.0,4.0,...,6.0,6.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0
2,5.0,2.0,27.0,5.0,3.0,400.0,1.0,1994.0,0.0,0.0,...,4.0,4.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0
3,4.0,1.0,20.0,6.0,12.0,400.0,3.0,1994.0,9.0,4.0,...,4.0,2.0,1.0,0.0,1.0,2.0,0.0,0.0,1.0,0.0
4,1.0,3.0,36.0,7.0,14.0,400.0,1.0,1994.0,1.0,5.0,...,6.0,5.0,1.0,0.0,1.0,1.0,0.0,0.0,1.0,0.0


In [12]:
from h2o.grid.grid_search import H2OGridSearch

# select the values for `balance_classes` to grid over
gbm_params2 = {'learn_rate': [0.01, 0.02, 0.03, 0.04, 0.05, 0.06, 0.07, 0.08, 0.09, 0.1],
                'max_depth': [2, 3, 4, 5, 6, 7, 8, 9, 10],
                'sample_rate': [0.5, 0.6000000000000001, 0.7000000000000001, 0.8, 0.9, 1.0],
                'col_sample_rate': [0.1, 0.2, 0.30000000000000004, 0.4, 0.5, 0.6000000000000001, 0.7000000000000001, 0.8, 0.9, 1.0]}

search_criteria = {"strategy": "RandomDiscrete", "max_models": 36, "seed": 1}

# this example uses cartesian grid search because the search space is small
# and we want to see the performance of all models. For a larger search space use
# random grid search instead: {'strategy': "RandomDiscrete"}
# initialize the GBM estimator

gbm_grid2 = H2OGridSearch(model=H2OGradientBoostingEstimator,
                          grid_id='gbm_grid2',
                          hyper_params=gbm_params2,
                          search_criteria=search_criteria)

gbm_grid2.train(x=predictors, y=response,
                training_frame=train,
                validation_frame=valid,
                ntrees=100,
                seed=1)
gbm_gridperf2 = gbm_grid2.get_grid(sort_by='auc', decreasing=True)
gbm_gridperf2

best_gbm2 = gbm_gridperf2.models[0]

best_gbm_perf2 = best_gbm2.model_performance(test)

best_gbm_perf2.auc() 

AttributeError: 'DataFrame' object has no attribute 'names'

In [ ]:
gbm_grid2.confusion_matrix()

In [ ]:
from h2o.grid.grid_search import H2OGridSearch

estimator = H2ORandomForestEstimator(
    # Stops fitting new trees when 10-tree rolling average is within 0.00001
    stopping_rounds = 10,
    stopping_tolerance = 0.00001,
    stopping_metric = 'auto',
    score_each_iteration = True,
    balance_classes = True,
    seed = 7)

hyper_parameters = {'ntrees':[5, 10, 20, 30, 40, 50, 60, 70, 80, 90, 100], 
                    'max_depth':[5, 10, 20, 30, 40, 50, 60, 70, 80, 90, 100]}

criteria = {"strategy": "RandomDiscrete", 
            "stopping_rounds": 10,
            "stopping_tolerance": 0.00001,
            "stopping_metric": "misclassification"}

grid_search = H2OGridSearch(model = estimator, 
                            hyper_params = hyper_parameters,
                            search_criteria = criteria)

In [ ]:
grid_search.train(x=predictors, y=response,
                training_frame=train,
                validation_frame=valid)

In [ ]:
gbm_gridperf2 = grid_search.get_grid(sort_by='auc', decreasing=True)
gbm_gridperf2

best_gbm2 = gbm_gridperf2.models[0]

best_gbm_perf2 = best_gbm2.model_performance(test)

best_gbm_perf2.auc() 

In [ ]:
grid_search.confusion_matrix()